In [16]:
import math
import sys
from image_positions import pos_dict, order_2_400
from PIL import Image, ImageFilter
import glob

import numpy as np
from numpy import random
from tqdm import tqdm

# First step
- Create a blank* image without number and croped images for all the numbers from the briefcases.
- Create a script to create a bunch of training samples to know where the biggest number is. (Should we find all the numbers?)
    - How do we write to an image?
- Store the images inside a folder, remember the order when created and the desired output of the model.

## Obtaining the numbers from the briefcases

As a convention, these are place number for each briefcase 

$$
\begin{pmatrix}
  0 & 1 & 2 & 3\\
  4 & 5 & 6 & 7\\
  8 & 9 & 10 & 11\\
  12 & 13 & 14 & 15
\end{pmatrix}
$$

Creating the sample images for the training:

In [ ]:
# Getting the cropped briefcase images
image_path = "../original_photos/clean_with_numbers_2-400.jpg"
image = Image.open(image_path).convert("RGB")

briefcase_list = [image.crop(pos_dict[x]) for x in range(16)]

current_combination = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

number_of_combinations = 1000
combinations_to_save = [random.permutation([y for y in range(16)]) for x in range(number_of_combinations)]

for current_combination in tqdm(combinations_to_save):

    for briefcase in range(16):
        image.paste(briefcase_list[current_combination[briefcase]], pos_dict[briefcase])

    file_name = "_".join([str(x) for x in current_combination])

    image.save(f"../training_data/2_400/{file_name}.png")

# Second step
- Train a model to know the position of the briefcases

## Training a model

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

2024-11-13 08:28:11.017131: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
# Lets get the list of files from training_data/2_400
dir_list = glob.glob("../training_data/2_400/*")
output_for_model = [list(map(int,training_dir[23:-4].split("_"))).index(2) for training_dir in dir_list] # .index(2) is to find where the 400 is

In [53]:
image_list = list()
for curr_image_dir in tqdm(dir_list):
    
    # Quick testing showed better results converting to black and white after filtering
    image = Image.open(dir_list[0])

    # image = image.convert('1')

    filtered = image.filter(ImageFilter.Kernel(
        size=(3, 3),
        kernel=[-1, -1, -1, -1, 8, -1, -1, -1, -1],
        scale=1
    ))

    filtered = filtered.convert('1')
    image_list.append(np.asarray(filtered))

  0%|          | 0/1000 [00:00<?, ?it/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
100%|██████████| 1000/1000 [00:55<00:00, 17.98it/s]


In [78]:
X_train, X_test, y_train, y_test = train_test_split(image_list, output_for_model, test_size=0.2)

In [79]:
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
X_train = X_train.reshape(
    X_train.shape[0], X_train.shape[1], X_train.shape[2], 1
)
X_test = X_test.reshape(
    X_test.shape[0], X_test.shape[1], X_test.shape[2], 1
)

In [82]:
# Create a convolutional neural network
model = tf.keras.models.Sequential([

    # Convolutional layer. Learn 32 filters using a 3x3 kernel
    tf.keras.layers.Conv2D(
        32, (3, 3), activation="relu", input_shape=(720, 1280, 1)
    ),

    # Max-pooling layer, using 2x2 pool size
    tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),

    # Flatten units
    tf.keras.layers.Flatten(),

    # Add a hidden layer with dropout
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),

    # Add an output layer with output units for all 10 digits
    tf.keras.layers.Dense(16, activation="softmax")
])

# Train neural network
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
model.fit(X_train, y_train, epochs=10)

# Evaluate neural network performance
model.evaluate(X_test,  y_test, verbose=2)

Epoch 1/10


: 